In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir("drive/MyDrive/Colab Notebooks/CS7650/final")
os.listdir()

Mounted at /content/drive


['data',
 'resiyrces.gdoc',
 'NBOW.ipynb',
 'Naive_Bayes.ipynb',
 'Clean_Data.ipynb',
 'Create_Vocab.ipynb']

In [ ]:
from collections import Counter
import pandas as pd
import numpy as np
import csv
import json

In [ ]:
df = pd.read_csv('data/train_clean.csv')
df.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0
1,1,000103f0d9cfb60f,daww he matches this background colour im seem...,0,0,0,0,0,0
2,2,000113f07ec002fd,hey man im really not trying to edit war its j...,0,0,0,0,0,0
3,3,0001b41b1c6bb37e,more i cant make any real suggestions on impro...,0,0,0,0,0,0
4,4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0,0,0,0,0,0


In [ ]:
class Vocab:
  def __init__(self, comments, min_count=0):
    self.num_words = 1
    self.word2id = {'UNK': 0}
    self.word_counts = Counter()
    self.min_count = min_count

    comments.str.split().apply(self.word_counts.update)
    for word in list(self.word_counts.keys()):
      if self.word_counts[word] > self.min_count:
        self.word2id[word] = self.num_words
        self.num_words += 1
    self.vocab = list(self.word2id.keys())
  
  def sentence2indices(self, sentences):
    return [[self.word2id.get(word, 0) for word in s] for s in sentences]
  
  def save_vocab(self):
    with open('data/vocab/word2id.json', 'w') as file:
      json.dump(self.word2id, file)
      file.close()
    with open('data/vocab/word_counts.json', 'w') as file:
      json.dump(self.word_counts, file)
      file.close()
    with open('data/vocab.json', 'w') as file:
      json.dump(self.vocab, file)
      file.close()

In [ ]:
V = Vocab(df['comment_text'])

In [ ]:
V.save_vocab()

In [ ]:
# def filter_glove(File, vocab):
#   print("Loading Glove Model")
#   f = open(File,'r')
#   gloveModel = {}
#   for line in f:
#     splitLines = line.split()
#     word = splitLines[0]
#     if word in vocab:
#       wordEmbedding = np.array([float(value) for value in splitLines[1:]])
#       gloveModel[word] = wordEmbedding
#   print(len(gloveModel)," words loaded!")
#   return gloveModel
def filter_glove(file, vocab):
  print('Loading gloVe')
  df = pd.read_table(file, sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)
  print(df.shape)
  print('filtering glove')
  df = df[df.index.isin(vocab)]
  print(df.shape)
  print('sending to dict')
  #glove = {key: val.values for key, val in df.T.items() if key in vocab}
  glove = {key: val.values.tolist() for key, val in df.T.items()}
  return glove

In [ ]:
glove = filter_glove('data/gloVe/glove.6B.300d.txt', V.vocab)
with open('data/gloVe/filtered_glove.json', 'w') as file:
  json.dump(glove, file)
  file.close()

Loading gloVe
(400000, 300)
filtering glove
(93979, 300)
sending to dict
